# **Exercise 1: Analyzing Energy Performance of Buildings**

## Overview

In this exercise, we analyze the [Energy Performance Certificate (EPC) dataset](https://epc.opendatacommunities.org/domestic/search) of individual dwellings in England and Wales. The EPC dataset records the *modelled* energy performance at the time of assessment, with each row corresponding to a single certificate issued following a standardised survey and calculation procedure, called [Standard Assessment Procedure (SAP)](https://www.gov.uk/guidance/standard-assessment-procedure). EPCs are model-based estimates under standard assumptions about weather, occupancy, and use, and *do not represent measured energy use*. EPC outputs are primarily designed for homeowners, renters, and policymakers.

By the end of this exercise, you should be able to:
- Process, visualize, and interpret building EPC data.
- Use descriptive/predictive analytics to identify the main factors that drive EPC ratings. 

## Set up

Load the required Python libraries and the data set. The data set contains information from 5000 buildings located in the London borough of Islington and is stored ```...\data raw\epc-certificates-Islington```; file ```certificates.csv``` contains the data and ```columns.csv``` explains the colunm headers. To load the data, update the ```path``` variable to the local folder location. 

For simplicity, columns with ```NaN``` values are dropped. An auxiliary function that estimates predictive accuracy, namely, mean squared error (MSE), root mean squared error (RMSE), and mean absolute error (MAE), is provided.

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def accuracy_metrics(actual, predictions):
    ''' Estimate predictive accuracy metrics '''
    actual_copy = actual.copy().reshape(-1,1)
    predictions_copy = predictions.copy().reshape(-1,1)
    
    error = actual_copy - predictions_copy    
    assert(error.shape[0] == len(actual_copy))
    if error.ndim > 1:        
        assert(error.shape[1] == 1)

    mse = np.square(error).mean()
    rmse = np.sqrt( mse )
    mae = np.abs(error).mean()
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    return rmse, mae

# default settings for plots
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (10,3)

# load data
# !!!! Update the path variable
path = 'C:\\Users\\ucbva19\\Git projects\\energy_analytics_built_env\\data raw\\epc-certificates-Islington'
df = pd.read_csv(f"{path}\\certificates.csv")  # change path

print(df.isna().sum())
display(df.head(3))

# drop columns with NaNs
df = df.dropna(axis=1)

assert(df.isna().sum().sum() == 0)

## Preliminary analysis

As an example of preliminary analysis, we generate a few informative plots and descriptive statistics.

In [ ]:
# Descriptive statistics
print(df['ENERGY_CONSUMPTION_CURRENT'].describe())
print(df['HEATING_COST_CURRENT'].describe())
print(df['CURRENT_ENERGY_RATING'].describe())

# Plot distribution
plt.figure()
plt.hist(df["ENERGY_CONSUMPTION_CURRENT"], bins=200, density=True)
plt.axvline(df["ENERGY_CONSUMPTION_CURRENT"].mean(), linestyle="--", color = 'black')
plt.xlim(0, 3000)
plt.title("ENERGY_CONSUMPTION distribution")
plt.show()


# EPC rating counts
plt.figure()
for k, g in df.groupby("CURRENT_ENERGY_RATING"):
    plt.hist(g["ENERGY_CONSUMPTION_CURRENT"], bins=150, histtype="step", label=str(k))
plt.xlim(0, df["ENERGY_CONSUMPTION_CURRENT"].quantile(0.99))
plt.title("Consumption grouped by EPC rating")
plt.legend(ncol=2, fontsize=8)
plt.show()

# Boxplot by EPC
plt.figure()
cats = sorted(df["CURRENT_ENERGY_RATING"].dropna().unique())
data = [df.loc[df["CURRENT_ENERGY_RATING"] == c, "ENERGY_CONSUMPTION_CURRENT"].values for c in cats]
plt.boxplot(data, labels = cats, showfliers=False)
plt.title("Consumption by EPC rating (boxplot)")
plt.xlabel("EPC")
plt.ylabel("ENERGY_CONSUMPTION $(kWh/m^2)$")
plt.show()

## Which factors drive EPC ratings?

Our main task is to analyze the factors that drive EPC ratings. We will use ```CURRENT_ENERGY_EFFICIENCY``` as the target/ outcome variable, which is a dimensionless variable that takes values in the range $(0,100)$, where $100$ indicates the most efficient dwelling (effectively, zero energy costs).

You may use any descriptive or predictive analytics method that you consider appropriate. There are multiple ways to tackle this problem. For instance, you can apply linear regression analysis and conduct statistical tests or train predictive machine learning (ML) models and calculate feature importance metrics, such as Shapley values.

#### Avoiding data leakage

Several variables in this dataset are model outputs, i.e., they are computed from the model itself. Using them as predictors should be avoided as it creates data leakage and circular logic. It is suggested to focus on features that relate to building characteristics, such as built form, fabric, services, and so forth. 

Here is a list of recommended features (feel free to include more):
- Built form: ```TOTAL_FLOOR_AREA```, ```PROPERTY_TYPE```, ```BUILT_FORM```, and ```FLOOR_HEIGHT```
- Envelope: ```WALLS_ENERGY_EFF``` and ```WINDOWS_ENERGY_EFF```
- Heating and hot water: ```MAINHEAT_ENERGY_EFF``` and ```HOT_WATER_ENERGY_EFF```
- Lighting and auxiliary: ```LIGHTING_ENERGY_EFF```, ```FIXED_LIGHTING_OUTLETS_COUNT``` and ```LOW_ENERGY_LIGHTING```

Note that variables such as ```WALLS_ENERGY_EFF``` or ```MAINHEAT_ENERGY_EFF``` are categorical SAP-derived efficiency bands (e.g. Poor / Average / Good), not measured efficiencies.

Variables about efficiency, rating, costs, or counterfactuals (marked with ```POTENTIAL```) *should not* be used as predictors. For instance, do not use ```CURRENT_ENERGY_EFFICIENCY, CURRENT_ENERGY_RATING, ENERGY_CONSUMPTION_CURRENT, CO2_EMISSIONS_CURRENT, ENVIRONMENT_IMPACT_CURRENT, HEATING_COST_CURRENT, HOT_WATER_COST_CURRENT, LIGHTING_COST_CURRENT, POTENTIAL_ENERGY_EFFICIENCY, POTENTIAL_ENERGY_RATING, ENERGY_CONSUMPTION_POTENTIAL``` and ```CO2_EMISSIONS_POTENTIAL```.

In [ ]:
target_variable = 'CURRENT_ENERGY_EFFICIENCY'
feature_list = ['TOTAL_FLOOR_AREA', 'PROPERTY_TYPE', 'BUILT_FORM',
                'WALLS_ENERGY_EFF', 'WINDOWS_ENERGY_EFF', 
                'MAINHEAT_ENERGY_EFF', 'HOT_WATER_ENERGY_EFF',
                'LIGHTING_ENERGY_EFF', 'FIXED_LIGHTING_OUTLETS_COUNT', 'LOW_ENERGY_LIGHTING']

Y = df[target_variable]
X = df[feature_list]

# print out categories for categorical features
for c in feature_list:
    if pd.api.types.is_string_dtype(df[c]): 
        print(f'Feature: {c}')
        print(f'Categories: {X[c].unique()}')

### Solution: Tree-based regression + Permutation Importance

We approach this problem using an ML predictor and estimating the importance of each feature. For this example, we choose the ```ExtraTreesRegressor``` predictor from the ```sklearn``` library and estimate the [Permutation Importance](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance) metric, which is a modeling inspection method that works with any trained predictor. An illustrative example and explanation can be found [here](https://scikit-learn.org/stable/modules/permutation_importance.html). In short, given a trained model, each feature (column) observation is randomly shuffled (permutated) and predictive performance is evaluated on a hold-out set. The shuffling process resembles adding noise and renders each feature effectively useless; the expected decrease in predictive accuracy is a good proxy for the importance of each feature.

The code below implements the following steps:
- Features are seperated into numerical, categorical, and ordinal features. Categorical features are one-hot encoded using ```OneHotEncoder``` and ordinal features are encoded using ```OrdinalEncoder```.
- To streamline the preprocessing, a ```ColumnTransformer``` and a ```Pipeline``` object are created.
- Data are split into train/test set and the ExtraTree regressor is trained.
- The Permutation Importance method is applied.

Tree-based methods estimate an internal feature importance metric during training, based on expected error reduction when splitting an internal node, which is plotted as well.  However, this metric is known to be biased. Comparing the two plots, we observe that the most important feature differs from the one obtained using the Permutation Importance metric.

In [ ]:
#%% Step 1: Pre-process and fit regression model

# Load libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.inspection import permutation_importance

target_variable = 'CURRENT_ENERGY_EFFICIENCY'

# further separate features into numerical, categorical, and ordinal
numerical_features = ['TOTAL_FLOOR_AREA', 'FIXED_LIGHTING_OUTLETS_COUNT', 'LOW_ENERGY_LIGHTING']
ordinal_features = ['WALLS_ENERGY_EFF', 'MAINHEAT_ENERGY_EFF', 'LIGHTING_ENERGY_EFF', 'HOT_WATER_ENERGY_EFF', 'WINDOWS_ENERGY_EFF']
categorical_features = ['PROPERTY_TYPE', 'BUILT_FORM']

feature_list = numerical_features + ordinal_features + categorical_features

Y = df[target_variable]
X = df[feature_list]

# Training/ test split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.33, random_state=42)

# Preprocessing
categories_list = [df['PROPERTY_TYPE'].unique(), df['BUILT_FORM'].unique()]

ord_list =   [['Very Poor', 'Poor', 'Average', 'Good', 'Very Good'] for feat in ordinal_features]

# create a preprocessor that implements one-hot and ordinal encoding
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
                  ("ord", OrdinalEncoder(categories = ord_list), ordinal_features),
                  ("num", "passthrough", numerical_features),])

# ExtraTree Model pipeline
et_model = Pipeline(
    steps=[("preprocess", preprocessor), ("regressor", ExtraTreesRegressor()),])

et_model.fit(train_X, train_Y)
y_et_pred = et_model.predict(test_X)

# evaluate accuracy
print('Evaluate model on out-of-sample data')
_, _ = accuracy_metrics(test_Y.values, y_et_pred)

# Permutation importance
r = permutation_importance(et_model, test_X, test_Y, n_repeats=30, random_state=0)
sorted_importances_idx = r.importances_mean.argsort()
importances = pd.DataFrame( r.importances[sorted_importances_idx].T, columns = X.columns[sorted_importances_idx],)

ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
plt.show()

# Feature importance (default estimation in tree-based algorithms)
# get feature names after one-hot encoding
feature_names = et_model.named_steps["preprocess"].get_feature_names_out()
importances = et_model.named_steps["regressor"].feature_importances_

fi = pd.Series(importances, index = feature_names).sort_values(ascending=False)

plt.figure(figsize=(10, 4))
fi.head(15).plot(kind="barh")
plt.gca().invert_yaxis()
plt.title("Feature importance (ExtraTrees)")
plt.xlabel("Importance")
plt.show()

Both approaches indicate that ```WALLS_ENERGY_EFF``` is the most important feature. Our results corroborate findings from existing studies. For instance, [1] finds that external wall U-value and dwelling geometry account for 75% of the variance of the energy rating of gas central-heated houses in England.

#### References

1. Stone, A., Shipworth, D., Biddulph, P. and Oreszczyn, T., 2014. Key factors determining the energy rating of existing English houses. Building Research & Information, 42(6), pp.725-738.